In [1]:
# This is a test

In [2]:
import pandas as pd
import numpy as np

df_AA2024 = pd.read_excel('data/filtered_AA2024.xlsx')

In [3]:
print(df_AA2024.describe())

           Time_h          pH  Inhib_Concentrat_M   Efficiency
count  611.000000  611.000000          611.000000   611.000000
mean   135.801964    6.342062            0.006808    26.736841
std    201.683867    2.529080            0.014059   288.788317
min      0.500000    0.000000            0.000010 -4834.000000
25%     24.000000    4.000000            0.000500    30.000000
50%     24.000000    7.000000            0.001000    58.000000
75%    144.000000    7.000000            0.003000    87.950000
max    672.000000   10.000000            0.100000   100.000000


In [4]:
# This is another test